SVM Base Model

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf # Version 1.0.0 (some previous versions are used in past commits)
tf.compat.v1.set_random_seed(42)
import matplotlib
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
import os
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout


session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)


sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

Using TensorFlow backend.


In [2]:
#read the dataset

#tf.compat.v1.disable_eager_execution()
from pandas import read_csv 
import pandas as pd
X_train = pd.read_csv('predictionDataset')
df = X_train.copy()


In [3]:
#convert categorical values to binary 
dummiesdf = pd.get_dummies(df['VisitorType'])
df.drop('VisitorType', inplace = True, axis = 1)
df['New_Visitor'] = dummiesdf['New_Visitor']
df['Other'] = dummiesdf['Other']
df['Returning_Visitor'] = dummiesdf['Returning_Visitor']
dfmonth = pd.get_dummies(df['Month']) 
df.drop('Month', inplace = True, axis = 1)
dfwithdummies = pd.concat([df, dfmonth], axis = 1, sort = False)
dfwithdummies['Class'] = df['Revenue'].astype(int)
dfwithdummies.drop('Revenue', axis = 1, inplace = True)
dfwithdummies['Weekend'] = df['Weekend'].astype(int)
dfwithdummies.drop('Returning_Visitor', axis = 1, inplace = True)
dfcleaned = dfwithdummies.copy()


X = dfcleaned.drop('Class', axis = 1)
Y = dfcleaned['Class'].copy()

#binary classification
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [4]:
#feature ranking and feature selection
from sklearn import svm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
list_one =[]
feature_ranking = SelectKBest(chi2, k=5)
fit = feature_ranking.fit(X, Y)

fmt = '%-8s%-20s%s'

for i, (score, feature) in enumerate(zip(feature_ranking.scores_, X.columns)):
    list_one.append((score, feature))
    
dfObj = pd.DataFrame(list_one) 
dfObj.sort_values(by=[0], ascending = False)

,0,1
5,877404.339415,ProductRelated_Duration
8,175126.808512,PageValues
1,41754.836841,Administrative_Duration
3,35059.775770,Informational_Duration
4,19317.285376,ProductRelated
0,1133.965531,Administrative
2,357.981605,Informational
24,223.548231,Nov
15,115.339482,New_Visitor
23,54.997108,May


In [5]:
#retaining only essential features
X.drop(['Nov','New_Visitor','May','SpecialDay','Mar','BounceRates','ExitRates','Feb','Oct','Dec','Browser','Weekend','June','Sep','Region','Aug','TrafficType','OperatingSystems','Other','Jul'],axis=1,inplace=True)


In [6]:
#converting to binary-valued features
X.ProductRelated[X.ProductRelated != 0] = 1
X.ProductRelated[X.ProductRelated == 0] = 0

X.Informational[X.Informational != 0] = 1
X.Informational[X.Informational == 0] = 0


X.Administrative[X.Administrative != 0] = 1
X.Administrative[X.Administrative == 0] = 0
X

C:\MyProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\MyProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\MyProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\MyP

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,PageValues
0,0,0.0,0,0.0,1,0.000000,0.000000
1,0,0.0,0,0.0,1,64.000000,0.000000
2,0,0.0,0,0.0,1,0.000000,0.000000
3,0,0.0,0,0.0,1,2.666667,0.000000
4,0,0.0,0,0.0,1,627.500000,0.000000
...,...,...,...,...,...,...,...
12325,1,145.0,0,0.0,1,1783.791667,12.241717
12326,0,0.0,0,0.0,1,465.750000,0.000000
12327,0,0.0,0,0.0,1,184.250000,0.000000
12328,1,75.0,0,0.0,1,346.000000,0.000000


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,onehot_encoded,test_size=.2)

In [8]:
####changing y_train from numpy array to df
y_train0=np.reshape(y_train,newshape=-1,order='F')
y_train_df = pd.DataFrame(data=y_train0[:9864],columns=["0"])
y_train_df["1"]=pd.DataFrame(data=y_train0[9864:])

In [9]:
####changing y_test from numpy array to df
y_test0=np.reshape(y_test,newshape=-1,order='F')
y_test_df = pd.DataFrame(data=y_test0[:2466],columns=["0"])
y_test_df["1"]=pd.DataFrame(data=y_test0[2466:])

In [10]:
def measure_network(prediction, actual):
    from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score, mean_squared_error
    print('Accuracy Score : ' + str(accuracy_score(prediction, actual)))
    print('Precision Score : ' + str(precision_score(prediction, actual)))
    print('Recall Score : ' + str(recall_score(prediction, actual)))
    print('F1 Score : ' + str(f1_score(prediction, actual)))
    print("The MSE is : " + str(mean_squared_error(prediction, actual)))

In [13]:
from sklearn.svm import SVC 
model = SVC() 
model.fit(X_train, y_train_df['0'])
predictedData0 = model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test_df['0'], predictedData0))
        

Accuracy Score : 0.8353609083536091
Precision Score : 0.9995133819951338
Recall Score : 0.83529890199268
F1 Score : 0.9100575985821887
The MSE is : 0.16463909164639093
              precision    recall  f1-score   support

         0.0       0.86      0.01      0.03       411
         1.0       0.84      1.00      0.91      2055

    accuracy                           0.84      2466
   macro avg       0.85      0.51      0.47      2466
weighted avg       0.84      0.84      0.76      2466

